# omfit_classes v. tokamaker read_eqdsk

This notebook compares a few geometry parameters extracted from the same eqdsk file with `omfit_classes.omfit_eqdsk.OMFITgeqdsk` and `OpenFUSIONToolkit.TokaMaker.util.read_eqdsk` methods. 

`omfit_classes` requires an older version of `scipy` and `numpy` so to use this notebook you have to run the following (if you dont use `uv` just leave that out):
```
uv pip install "scipy==1.13.1"
```

and 
```
uv pip install "numpy<2.0"
```

When you're done and you want to upgrade `scipy` and `numpy` back to the latest version, run this:
```
uv pip install --update scipy numpy
```

The two report exactly the same values for what is available from both, so I am moving forward with trusting `Tokamaker.read_eqdsk`. (Note if you have the TokaMaker object after running `mygs.solve()` there are more geometry parameters available, they also seem to match what `omfit_classes` extracts from the eqdsk file.)


F. Sheehan 2026-02-15


In [1]:
from OpenFUSIONToolkit.TokaMaker.util import read_eqdsk
import matplotlib.pyplot as plt
from omfit_classes.omfit_eqdsk import OMFITgeqdsk
import numpy as np


In [2]:
# ===== Load OMFIT data into dict =====
g = OMFITgeqdsk('iter_hmode.eqdsk')
g.load()

# Load flux surfaces for geometry calculations
g['fluxSurfaces'].load()
fs = g['fluxSurfaces']

# Populate OMFIT dictionary
omfit = {}

# Basic parameters
omfit['psi_axis'] = g['SIMAG']
omfit['psi_lcfs'] = g['SIBRY']
omfit['R0'] = g['RCENTR']
omfit['B0'] = g['BCENTR']
omfit['Ip'] = g['CURRENT']

# Q profile
omfit['q_profile'] = g['QPSI']
psi_grid = np.linspace(omfit['psi_axis'], omfit['psi_lcfs'], len(omfit['q_profile']))
omfit['psi_N'] = (psi_grid - omfit['psi_axis']) / (omfit['psi_lcfs'] - omfit['psi_axis'])

# Flux surface data
omfit['volume_vs_psi_N'] = fs['geo']['vol']
omfit['psi_N_fs'] = fs['geo']['psin']
omfit['volume_lcfs'] = omfit['volume_vs_psi_N'][-1]

# Calculate <1/R> and <1/R²> manually
inv_R_list = []
inv_R2_list = []
for i, psin_val in enumerate(omfit['psi_N_fs']):
    R_surf = fs['R'][i] if 'R' in fs else None
    if R_surf is not None:
        inv_R_list.append(np.mean(1.0 / R_surf))
        inv_R2_list.append(np.mean(1.0 / R_surf**2))

omfit['inv_R_avg'] = np.array(inv_R_list) if inv_R_list else None
omfit['inv_R2_avg'] = np.array(inv_R2_list) if inv_R2_list else None

print("OMFIT data loaded successfully")
print(f"ψ_axis = {omfit['psi_axis']:.4f} Wb/rad")
print(f"ψ_lcfs = {omfit['psi_lcfs']:.4f} Wb/rad")
print(f"q(axis) = {omfit['q_profile'][0]:.3f}")
print(f"q(edge) = {omfit['q_profile'][-1]:.3f}")


OMFIT data loaded successfully
ψ_axis = -19.2763 Wb/rad
ψ_lcfs = -10.0177 Wb/rad
q(axis) = 1.380
q(edge) = 4.207


In [9]:
# ===== Load TokaMaker data into dict =====
tm_data = read_eqdsk('iter_hmode.eqdsk')

# Populate TokaMaker dictionary with same structure as OMFIT
tm = {}

# Basic parameters
tm['psi_axis'] = tm_data['psimag']
tm['psi_lcfs'] = tm_data['psibry']
tm['R0'] = tm_data['rcentr']
tm['B0'] = tm_data['bcentr']
tm['Ip'] = tm_data['ip']

# Q profile
tm['q_profile'] = tm_data['qpsi']
psi_grid_tm = np.linspace(tm['psi_axis'], tm['psi_lcfs'], len(tm['q_profile']))
tm['psi_N'] = (psi_grid_tm - tm['psi_axis']) / (tm['psi_lcfs'] - tm['psi_axis'])

# Note: TokaMaker may not have flux surface geometry automatically computed
# Check what's available in tm_data
print(f"psi_axis = {tm['psi_axis']:.4f} Wb/rad")
print(f"psi_lcfs = {tm['psi_lcfs']:.4f} Wb/rad")
print(f"q_axis = {tm['q_profile'][0]:.3f}")
print(f"q_lcfs = {tm['q_profile'][-1]:.3f}")
print(f"\nAvailable keys in TokaMaker data: {list(tm_data.keys())}")


psi_axis = -19.2763 Wb/rad
psi_lcfs = -10.0177 Wb/rad
q_axis = 1.380
q_lcfs = 4.207

Available keys in TokaMaker data: ['case', 'nr', 'nz', 'rdim', 'zdim', 'rcentr', 'rleft', 'zmid', 'raxis', 'zaxis', 'psimag', 'psibry', 'bcentr', 'ip', 'fpol', 'pres', 'ffprim', 'pprime', 'psirz', 'qpsi', 'nbbs', 'nlim', 'rzout', 'rzlim']


In [15]:
print(f"\n{'Parameter':<20} {'OMFIT':<20} {'TokaMaker':<20} {'Diff %':<15}")
print("-"*75)

# Basic parameters
params = [
    ('psi_axis (Wb/rad)', omfit['psi_axis'], tm['psi_axis']),
    ('psi_lcfs (Wb/rad)', omfit['psi_lcfs'], tm['psi_lcfs']),
    ('R0 (m)', omfit['R0'], tm['R0']),
    ('B0 (T)', omfit['B0'], tm['B0']),
    ('Ip (A)', omfit['Ip'], tm['Ip']),
    ('q_axis', omfit['q_profile'][0], tm['q_profile'][0]),
    ('q_lcfs', omfit['q_profile'][-1], tm['q_profile'][-1]),
]

if omfit.get('volume_lcfs') is not None:
    params.append(('Volume LCFS (m³)', omfit['volume_lcfs'], None))

for name, val_omfit, val_tm in params:
    if val_tm is not None:
        diff_pct = 100 * (val_omfit - val_tm) / val_omfit if val_omfit != 0 else 0
        print(f"{name:<20} {val_omfit:<20.6g} {val_tm:<20.6g} {diff_pct:>14.4f}%")
    else:
        print(f"{name:<20} {val_omfit:<20.6g} {'N/A':<20} {'N/A':<15}")




Parameter            OMFIT                TokaMaker            Diff %         
---------------------------------------------------------------------------
psi_axis (Wb/rad)    -19.2763             -19.2763                    -0.0000%
psi_lcfs (Wb/rad)    -10.0177             -10.0177                    -0.0000%
R0 (m)               6.22621              6.22621                      0.0000%
B0 (T)               5.36281              5.36281                      0.0000%
Ip (A)               1.3e+07              1.3e+07                      0.0000%
q_axis               1.37993              1.37993                      0.0000%
q_lcfs               4.20657              4.20657                      0.0000%
Volume LCFS (m³)     809.791              N/A                  N/A            
